In [30]:
import numpy as np
import pandas as pd
import tables

In [31]:
data = pd.read_hdf('data.h5')

In [38]:
cal = pd.read_csv('calendar.csv')

In [33]:
sales = pd.read_csv('sales_train_validation.csv')

In [34]:
# week day feature
cal_ = cal.loc[:,['wday','month','event_name_1','snap_CA','snap_TX','snap_WI']]
cal_['day_type'] = cal_['wday'].map({1: 5,2:6,3: 0,4:2,5:2,6:2,7:4})
cal_.loc[cal_['event_name_1'].isna(), 'holi_bool'] = 0
cal_.loc[~cal_['event_name_1'].isna(), 'holi_bool'] = 1
cal_['holi_-1'] = cal_["holi_bool"].shift(1)
cal_['holi_1'] = cal_["holi_bool"].shift(-1)
cal_.loc[cal_['holi_-1']==1, 'day_type'] = 8
cal_.loc[cal_['holi_1']==1, 'day_type'] = 8
cal_.loc[cal_['holi_bool']==1, 'day_type'] = 7
cal_.drop(['wday','month','event_name_1','holi_bool','holi_-1','holi_1'], axis=1, inplace=True)

In [35]:
cal_.to_csv('./cal_feat.csv',index=False)

In [37]:
cal_.T

,0,1,2,3,4,5,6,7,8,9,...,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968
snap_CA,0,0,0,1,1,1,1,1,1,1,...,1,0,0,0,0,0,0,0,0,0
snap_TX,0,0,0,1,0,1,0,1,1,1,...,0,1,1,1,0,1,0,0,0,0
snap_WI,0,0,0,0,1,1,0,1,1,0,...,0,1,1,0,1,1,0,0,0,0
day_type,5,6,0,2,2,2,4,8,7,8,...,4,5,6,0,2,2,2,4,8,7


In [9]:
price_df = pd.read_csv('sell_prices.csv')
cal_df = pd.read_csv('calendar.csv')
sales = pd.read_csv('sales_train_validation.csv')
store_list = price_df['store_id'].unique().tolist()
for store in store_list:
    p_df = pd.merge(price_df, cal_df.loc[:,['wm_yr_wk', 'd']], on='wm_yr_wk')
    p_df = p_df[p_df['store_id']==store].loc[:,['item_id','sell_price','d']]
    p_df = p_df.set_index(['item_id','d'])
    p_df = p_df.unstack(level='d').rename(columns= lambda x: x[2:])
    levels = p_df.columns.levels
    labels = p_df.columns.labels
    p_df.columns = levels[1][labels[1]]
    new_idx=[str(i+1) for i in range(1969)]
    p_df = p_df.loc[:,new_idx]
    p_df = pd.merge(sales.loc[:,'item_id'], p_df, on='item_id', how='left')
    p_df = p_df.fillna(0)
    p_df.to_csv(f"./price_{store}.csv",index=False)

/Users/shukigoto/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  # This is added back by InteractiveShellApp.init_path()


In [7]:
p_df.fillna(0)

,item_id,1,2,3,4,5,6,7,8,9,...,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
0,HOBBIES_1_001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38
1,HOBBIES_1_002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97
2,HOBBIES_1_003,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97
3,HOBBIES_1_004,4.34,4.34,4.34,4.34,4.34,4.34,4.34,4.34,4.34,...,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64
4,HOBBIES_1_005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823,2.50,2.50,2.50,2.50,2.50,2.50,2.50,2.50,2.50,...,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98
30486,FOODS_3_824,2.47,2.47,2.47,2.47,2.47,2.47,2.47,2.47,2.47,...,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48
30487,FOODS_3_825,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,...,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98
30488,FOODS_3_826,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28


In [27]:
price_df = pd.read_csv('sell_prices.csv')
cal_df = pd.read_csv('calendar.csv')
sales = pd.read_csv('sales_train_validation.csv')
store_list = price_df['store_id'].unique().tolist()
for store in store_list:
    p_df = pd.merge(price_df, cal_df.loc[:,['wm_yr_wk', 'd']], on='wm_yr_wk')
    p_df = p_df[p_df['store_id']==store].loc[:,['item_id','sell_price','d']]
    p_df = p_df.set_index(['item_id','d'])
    p_df = p_df.unstack(level='d').rename(columns= lambda x: x[2:])
    levels = p_df.columns.levels
    labels = p_df.columns.labels
    p_df.columns = levels[1][labels[1]]
    new_idx=[str(i+1) for i in range(1969)]
    p_df = p_df.loc[:,new_idx]
    p_df = pd.merge(sales[sales['store_id']==store].loc[:,'item_id'], p_df, on='item_id', how='left')
    p_df = p_df.fillna(0)
    p_df.to_csv(f"./price_{store}.csv",index=False)

/Users/shukigoto/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  # This is added back by InteractiveShellApp.init_path()


In [28]:
p_df

,item_id,1,2,3,4,5,6,7,8,9,...,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969
0,HOBBIES_1_001,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38
1,HOBBIES_1_002,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97
2,HOBBIES_1_003,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97
3,HOBBIES_1_004,4.34,4.34,4.34,4.34,4.34,4.34,4.34,4.34,4.34,...,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64
4,HOBBIES_1_005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_823,2.50,2.50,2.50,2.50,2.50,2.50,2.50,2.50,2.50,...,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98
3045,FOODS_3_824,2.47,2.47,2.47,2.47,2.47,2.47,2.47,2.47,2.47,...,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48
3046,FOODS_3_825,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,...,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98
3047,FOODS_3_826,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28
